In [56]:
!pip install langchain langchain_openai langchain_experimental pandas numpy tabulate openpyxl -q

In [57]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import numpy as np
import os

In [61]:
df = pd.read_csv("Titanic.csv") #El Data set esta asociado a las persona que estuvieron en el Titanic
df.head() #Confirmamos estructura y lectura del CSV.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
Api_Key = os.getenv('OPENAI_API_KEY') #Usamos dotenv para almacenar la clave de la API con la que nos conectamos a OPENAI.
os.environ["OPENAI_API_KEY"] = Api_Key #Pasamos la clave de la API de OpenAI
# os.environ["OPENAI_API_KEY"] = "TuClave de OPENAI" Tambien podemos usar esta linea por si no usamos dotenv
model = "gpt-3.5-turbo-16k" #Seleccionamos el modelo a utilizar
llm_model = ChatOpenAI(model = model, temperature=0, verbose = True, streaming=True) #Creamos el modelo.

## Escenario 1. Consultamos sin usar Data Agent de Pandas.
La desventaja de esta metodo, es que ante preguntas complejas generar error. Funciona para consultas sencilla

In [62]:
#Creamos el prompt
df_parser = PandasDataFrameOutputParser(dataframe=df)
prompt= PromptTemplate (
    template = "Responde al usuario su consulta. \n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions": df_parser.get_format_instructions()},
)

chain = prompt | llm_model | df_parser

In [63]:
# Hacemos la pregunta a ser respondida por el modelo. Y nos da una respuesta correcta.
chain.invoke({
    "query":"dame la primera fila."
})

{'0': PassengerId                          1
 Survived                             0
 Pclass                               3
 Name           Braund, Mr. Owen Harris
 Sex                               male
 Age                               22.0
 SibSp                                1
 Parch                                0
 Ticket                       A/5 21171
 Fare                              7.25
 Cabin                              NaN
 Embarked                             S
 Name: 0, dtype: object}

## Escnerario 2: Usamos el Data Agent de Pandas, para que ante preguntas complejas nos de respuestas correctas.
La ventaja de usar el Data Agent de pandas, es que sirve e intermedio entre el lenguaje natural y el modelo, enviandole intrucciones mas "precisas" al modelo.

In [65]:
#Creamos el Data Agent
pandas_df_agent = create_pandas_dataframe_agent(
    llm = llm_model, #modelo a usar
    df=df, #dataFrame
    verbose=True, #Mostrar en pantalla que realiza
    #handle_parsing_errors=True, #Tratamiento de errores
    agent_type = AgentType.OPENAI_FUNCTIONS, #El tipo de agente a usar
    allow_dangerous_code = True, #De debe agregar, ya que estamos ejecutando el modelo fuera del SandBox, no recomendado por seguridad.
    return_intermediate_steps=True) #Retorno de etapas intermedias.

In [66]:
#Hacemos la pregunta y su respuesta es acertada.
Pandas_result = pandas_df_agent.invoke("cual es la edad media de las mujeres")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Sex'] == 'female']['Age'].mean()"}`


27.915708812260537La edad media de las mujeres en el dataframe es de aproximadamente 27.92 años.

> Finished chain.


In [67]:
#Hacemos la pregunta y su respuesta es acertada.
Pandas_result = pandas_df_agent.invoke("Cuales son los nombres que las personas que sobrevivieron")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Survived'] == 1]['Name']"}`


1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2                                  Heikkinen, Miss Laina
3           Futrelle, Mrs. Jacques Heath (Lily May Peel)
8      Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                    Nasser, Mrs. Nicholas (Adele Achem)
                             ...                        
875                      Najib, Miss Adele Kiamie "Jane"
879        Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)
880         Shelley, Mrs. William (Imanita Parrish Hall)
887                          Graham, Miss Margaret Edith
889                                Behr, Mr. Karl Howell
Name: Name, Length: 342, dtype: objectLos nombres de las personas que sobrevivieron son:

1. Cumings, Mrs. John Bradley (Florence Briggs Thayer)
2. Heikkinen, Miss Laina
3. Futrelle, Mrs. Jacques Heath (Lily May Peel)
4. Johnson, Mrs. Oscar W (Elisab